In [ ]:
import re
import pandas as pd
from collections import defaultdict
import spacy
import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [ ]:
file_name = f"extracted_text_1.csv"
df = pd.read_csv(file_name, on_bad_lines='skip') ### Load the data ####
### filter out columns ###
df.columns = ['PMID', 'DOI', 'Authors', 'Title','Content']
df.head(10)

Replacing abbreviations

In [ ]:
import re

def replace_with_dict(sentence, my_dict):
    for key, value in my_dict.items():
        # Replace the key in the sentence with the value
        sentence = re.sub(r'\b' + re.escape(key) + r'\b', value, sentence)
    return sentence

In [ ]:
list1 = []
list2 = []
pattern = r'^\D+$'
for sentence in df["Content"]:
    my_dict = {}
    sent = ""
    # print(sentence)
    sentence = str(sentence)
    sentence = re.sub(r'[.,]', '', sentence)
    text_list = sentence.split()
    if "(" in sentence:
      for i in range(len(text_list)):
        elem = text_list[i][0]
        if text_list[i][0]=="(" and len(text_list[i]) <8 and ")" in text_list[i] and len(text_list[i]) >3 and re.match(pattern, text_list[i]):
          for k in range(i-2,i-5,-1):
            try:
                if text_list[k][0].lower() == text_list[i][1].lower():
                  for p in range(k,i):
                    sent += text_list[p]+" "
                  if sent not in list1:
                      list1.append(sent)
                      my_dict[text_list[i][1:len(text_list[i])-1]] = sent
                  sent = ""
                  break
            except:
              pass
    modified_sentence = replace_with_dict(sentence, my_dict)
    list2.append(modified_sentence)
df["abr"] = pd.Series(list2)

Using SpaCy to preprocess the text. Performing the following steps:

lowercase the words
remove the stopwords and single characters
use regex to remove non-alphabetic characters (anything that is not a number or alphabet including punctuations), in other words only keep "a" to "z" and digits.
remove lines that have less than 4 words, since they cannot contribute much to the training process.

In [ ]:
nlp = spacy.load("en_core_web_sm", disable=["tagger", "parser","ner"])
stopwords = spacy.lang.en.stop_words.STOP_WORDS
df["cleaned"] = pd.Series( str(quote).lower() for quote in df["abr"])
list0 = []
for sentence in df["cleaned"]:
    for word in stopwords:
        if "'" in word:
            sentence = re.sub(re.escape(word) + r'\b' , '', sentence, flags=re.IGNORECASE)
        else:
            sentence = re.sub(r'\b' + re.escape(word) + r'\b' , '', sentence, flags=re.IGNORECASE)
    list0.append(sentence)
df["cleaned"] = pd.Series(list0)
df["cleaned"] = pd.Series([re.sub(r'[^a-zA-Z0-9\s\(\)]', "", quote) for quote in df["cleaned"]])# lowercase and remove non-alphabetic characters
df["cleaned"] = pd.Series([re.sub(r"\b[a-zA-Z]\b", "", sentence) for sentence in df["cleaned"]])
df["cleaned"] = pd.Series([re.sub(r'\s+', ' ', text).strip() for text in df["cleaned"]])
def word_count(sentence):
    return len(sentence.split())

filtered_df = df[df["cleaned"].apply(word_count) >= 4]
print(filtered_df)

quotes =[sentence for sentence in filtered_df["cleaned"]] # to save all the lines
# print(quotes)

In [ ]:
del df["abr"]

Saving in a new file preprocessed code

In [ ]:
df.to_csv(f'file_{num}.csv', index=False)